In [6]:
import pandas as pd
import os
import re
import sys
import numpy as np
from merf import MERF
import matplotlib.pyplot as plt
import seaborn as sns
import itertools 
sns.set_context("poster")
from sklearn.ensemble import RandomForestRegressor
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = (11,8)
from merf.merf import MERF
from sklearn.model_selection import train_test_split, KFold
from merf.viz import plot_merf_training_stats
from sklearn.model_selection import KFold

current_dir = os.getcwd() # Get the current working directory
parent_dir = os.path.abspath(os.path.join(current_dir, '../'))
sys.path.append(parent_dir)
from em_utils import * # import the utils

In [ ]:
# Create output directory if it doesn't exist
# output_dir = "/Users/emily/projects/research/Stanislawski/comps/mutli-omic-predictions/play_scripts/2.models/merf_python/april/new_split"
output_dir = "/Users/emily/projects/research/Stanislawski/comps/mutli-omic-predictions/play_scripts/2.models/merf_python/may_basic_plus/delta/"
df_dir = "/Users/emily/projects/research/Stanislawski/comps/mutli-omic-predictions/play_scripts/2.models/merf_python/may_basic_plus/merf_dfs_delta/"
os.makedirs(df_dir, exist_ok=True)

print("---------- Read data ----------")
input_dir = "/Users/emily/projects/research/Stanislawski/comps/mutli-omic-predictions/data/april_processing/"
delta = read_data(input_dir, "all_delta_april29.csv")
print(delta.columns.to_list())

In [ ]:
#Make time numeric 
time_mapping = {
    "BL": 0,
    "6m": 6,
    "12m": 12
}
delta['time'] = delta['time'].replace(time_mapping)
print(delta['time'])

In [9]:
# Make omic subsets
BMI_VAR, ID_VAR, TIME_VAR, DATA = 'outcome_BMI_fnl', 'subject_id', 'time', delta
# Define the column names based on your lists
basic = [BMI_VAR, ID_VAR, TIME_VAR, 'sex', 'age', 'randomized_group']
meta_keep = [BMI_VAR, ID_VAR, TIME_VAR, 'randomized_group', 'sex', 'race', 
             'age', 'Glucose.x', 'HOMA_IR', 'Insulin_endo', 'HDL_Total_Direct_lipid', 'LDL_Calculated', 'Triglyceride_lipid']
only_grs = [BMI_VAR, ID_VAR, TIME_VAR, 'bmi_prs']
only_taxa = [BMI_VAR, ID_VAR, TIME_VAR,] + [col for col in DATA.columns if col.startswith("g__")]

micom_start = DATA.columns.get_loc("Diacetyl")
micom_end = DATA.columns.get_loc("aldehydo.D.xylose")
only_micom = [BMI_VAR, ID_VAR, TIME_VAR,] + list(DATA.columns[micom_start:micom_end + 1])

path_start = DATA.columns.get_loc("arginine..ornithine.and.proline.interconversion")
path_end = DATA.columns.get_loc("UDP.N.acetyl.D.glucosamine.biosynthesis.I")
only_pathway = [BMI_VAR, ID_VAR, TIME_VAR,] + list(DATA.columns[path_start:path_end + 1])

metabo_start = DATA.columns.get_loc("non_HDL_C")
metabo_end = DATA.columns.get_loc("IDL_TG_pct")
only_metabo = [BMI_VAR, ID_VAR, TIME_VAR,] + list(DATA.columns[metabo_start:metabo_end + 1])

all_col = [BMI_VAR, ID_VAR, TIME_VAR,] + ['randomized_group', 'sex', 'race',
    'age','Glucose.x', 'HOMA_IR', 'Insulin_endo', 'HDL_Total_Direct_lipid', 'LDL_Calculated', 'Triglyceride_lipid'] + \
    list(DATA.columns[DATA.columns.str.startswith("g__")]) + \
    list(DATA.columns[micom_start:micom_end + 1]) + \
    list(DATA.columns[path_start:path_end + 1]) + \
    list(DATA.columns[metabo_start:metabo_end + 1])

In [ ]:
# Make train and test sets 
# test sample names
#test_names = ["ABR-079", "AGA-071", "AHE-055", "ALI-121", "ALO-163", "AMA-031", "ASO-013", "AWI-167", "BMO-164", "CWA-183", "DSC-024", "EBE-130", "EHI-177", "EJO-092", "GFU-188", "HGI-010", "JCA-109", "JGO-100",
#    "KBU-085", "KCE-034", "KHE-170", "LDO-148", "LST-186", "LZD-142", "MAR-119", "MCA-088", "MJA-153", "MWE-112", "NPO-149", "RAE-114", "SBO-020", "SEG-080", "SKA-195", "SLO-178", "SSH-028", "TDU-086","TFA-016", "VCA-041"]
test_names = ["ASO-013", "NTA-021", "KGI-029", "KPA-042", "AWA-052", "AHE-055", "COW-066", "NBI-069", "CEL-073", "CAL-074", "ABR-079", "SEG-080", "NKA-090", "NEL-094", "LJA-101", "ADA-105", "MLU-106", "MDI-107", "JER-110", "TRO-113", "MFB-118", "ALI-121", "KWA-122", "RAF-125", "EBE-130", "CGA-134", "LZD-142", "NPO-149", "HDE-154", "AMC-155", "SAB-160", "QNG-166", "NCO-171", "BSA-174", "EHI-177", "LST-186", "MBA-187", "BAN-193"]
# train sample names
#train_names = ["AAL-144", "ACO-053", "ADA-105", "AKE-009", "AKI-011", "AKO-139", "AMC-155", "AME-128", "AME-157", "ATA-129", "AWA-052", "AWA-083", "BAN-193", "BHO-014", "BIN-201", "BKN-104", "BMI-156", "BSA-174", "CAM-057", "CCO-189",
#    "CED-026", "CEL-073", "CGA-134", "CIS-077", "CKR-078", "CLE-049", "COW-066", "CRO-108", "CWA-161", "EBE-051", "EKA-135", "EKR-045", "ELA-159", "EPO-182", "EVO-184", "FWI-098", "GHA-035", "HDE-154", "IBE-120", "JDI-140", "JER-110", "JFU-027", "JJO-093", "JKN-127", "JPO-022", "JUG-116", "JUT-032", "JVE-126", "KAN-138", "KBR-162", "KEL-185", "KEL-199", "KGI-029", "KHU-196", "KPA-042", "KRI-072", "KVA-038", "KWA-122", "KWA-141", "LBL-047", "LBU-015", "LEL-147", "LFI-003", "LJA-101", "LMC-111", "LPF-198", "LVA-017", "MBA-187", "MCW-065", "MDI-107", "MES-068", "MFB-118", "MGA-076", "MHO-117", "MKE-192", "MMA-036", "MRT-179", "MSH-091", "MST-039", "MWE-143",
#    "MWO-133", "MWY-152", "NAR-099", "NBI-048", "NBI-069", "NCO-171", "NDI-067", "NEL-094", "NKA-090", "NMO-151", "NTA-021", "PBE-123", "QNG-166", "RAF-125", "RAM-050", "RHP-023", "RLA-132", "ROL-006", "SAB-160", "SCA-043", "SCR-061", "SDA-150", "SGA-062", "SKA-087", "SRO-194", "TBU-115", "TFA-172", "TRO-113", "TSH-146", "TSL-056", "WPE-005", "YOR-103", "YSU-097", "ZVU-096"]
train_names = ["SDA-150", "LBU-015", "CIS-077", "ATA-129", "KHU-196", "MWY-152", "AGA-071", "AME-157", "CWA-183", "RHP-023", "MST-025", "SSH-028", "JUG-116", "EJO-092", "VCA-041", "NMO-151", "BHO-014", "KBU-085", "SBO-020", "MWO-133", "KRI-072", "AAL-144", "ALO-163", "AKI-011", "MHO-117", "TSH-146", "RAE-114", "FWI-098", "MAR-119", "JGO-100", "CAM-057", "YOR-103", "HGI-010", "KAN-138", "SGA-062", "CKR-078", "MWE-112", "ROL-006", "MMA-036", "DSC-024", "LDO-148", "MCA-088", "CPU-075", "AKO-139", "LFI-003", "KWA-141", "GFU-188", "BMO-164", "JPO-022", "EVO-184", "LPF-198", "TBU-115", "SRO-194", "KEL-199", "JFU-027", "SKA-195", "IBE-120", "TSL-056", "NDI-067", "AWA-083", "CWA-161", "TDU-086", "JCA-109", "CBO-004", "NAR-099", "MES-068", "AMA-031", "SLO-178", "SCA-043", "AWI-167",  "KBR-162", "TFA-172", "BIN-201", "NBI-048", "KHE-170", "CSH-012", "BMI-156", "MWE-143", "EKA-135", "WPE-005", "AKE-009", "YSU-097", "MCW-065", "EBE-051", "ZVU-096", "JJO-093", "KVA-038", "ACO-053", "RLA-132", "MBA-176", "CED-026", "JDI-140", "CCO-189", "EKR-045", "MJA-153", "CLE-049", "LMC-111", "SKA-087", "JUT-032", "MKE-192", "JVE-126", "KCE-034", "KEL-185", "MRT-179", "JKN-127", "LEL-147", "BKN-104", "AME-128", "MSH-091", "MGA-076", "LVA-017", "EPO-182"]

print("Length of test names:", len(test_names))
print("Length of train names:", len(train_names))

# make summary table 

In [ ]:
print(DATA.columns)
#print(delta.columns)
print([ID_VAR])

In [ ]:
column_sets = {
    "basic": basic,
    "meta_keep": meta_keep,
    "only_grs": only_grs, 
    "only_taxa": only_taxa,
    "only_micom": only_micom,
    "only_pathway": only_pathway,
    "only_metabo" : only_metabo,
    "only_all" : all_col
}

# Initialize a list to store summary data
summary_data = []

# Loop through each column set and create DATA
for key, columns in column_sets.items():
    DATA = delta[columns]
    print(f"Subset for {key} created with shape: {DATA.shape}")
    
    # Calculate summary statistics
    summary = DATA.groupby('time').agg(
    unique_id_count=(ID_VAR, 'nunique'),  # Count unique ID_VAR
    feature_count=('time', lambda x: DATA.shape[1] - 2)  # Count total features excluding 'time' and 'ID_VAR'
    ).reset_index()
    
    # Print the summary table
    print("\nSummary Table for", key)
    print(summary.to_string(index=False))  # Print without the index
    
    # Append summary to the summary_data list
    summary_data.append((key, summary))


In [ ]:
column_sets = {
    "basic": basic,
    "meta_keep": meta_keep,
    "only_grs": only_grs, 
    "only_taxa": only_taxa,
    "only_micom": only_micom,
    "only_pathway": only_pathway,
    "only_metabo": only_metabo,
    "only_all": all_col
}

for key, columns in column_sets.items():
    print(f"\n=== Processing column set: {key} ===")
    DATA = delta[columns]
    print(f"Subset for {key} created with shape: {DATA.shape}")

    OUT = f"/Users/emily/projects/research/Stanislawski/comps/mutli-omic-predictions/play_scripts/2.models/merf_python/may_basic_plus/delta/{key}_delta.csv"
    output_dir = "/Users/emily/projects/research/Stanislawski/comps/mutli-omic-predictions/play_scripts/2.models/merf_python/may_basic_plus/delta/"
    r2_out = f"{key}_delta_r2.pdf"
    r2_adj_out = f"{key}_delta_r2_adj.pdf"
    feature_imp_out = f"{key}_delta_ft_imp.pdf"
    results_filename = f"/Users/emily/projects/research/Stanislawski/comps/mutli-omic-predictions/play_scripts/2.models/merf_python/may_basic_plus/{key}_delta_april29.csv"

    # Train/test split
    train_set = DATA[DATA[ID_VAR].isin(train_names)]
    test_set = DATA[DATA[ID_VAR].isin(test_names)]

    clusters_train = pd.Series(train_set[ID_VAR])
    clusters_test = pd.Series(test_set[ID_VAR])

    print("---------- Select predictors for training set ----------")
    X = train_set.drop([BMI_VAR, ID_VAR], axis=1)
    Y = train_set[BMI_VAR].to_numpy()
    Z = np.ones((train_set.shape[0], 1))

    print(f"Length of X: {len(X)}, Length of clusters_train: {len(clusters_train)}, Length of Y: {len(Y)}")
    assert len(X) == len(clusters_train)
    assert len(X) == len(Y)
    print("Final columns after drop:", X.columns.to_list())

    print("---------- Select predictors for test set ----------")
    X_new = test_set.drop([BMI_VAR, ID_VAR], axis=1)
    X_new = X_new[X.columns]  # align with training set columns
    X_new = X_new.astype(X.dtypes)
    Y_new = test_set[BMI_VAR].to_numpy()
    clusters_new = pd.Series(test_set[ID_VAR])
    Z_new = np.ones((len(X_new), 1))
    time_new = test_set[TIME_VAR].astype(float).to_numpy()

    # Hyperparameters
    param_grid = {
        'n_estimators': [10, 50, 100],
        'max_depth': [None],
        'min_samples_split': [0.05, 0.1, 0.15],
        'max_iter': [2, 10],
        'n_splits': [3, 5, 10]
    }

    y = train_set[[BMI_VAR]][BMI_VAR].to_numpy()
    clusters = train_set[ID_VAR].to_numpy()
    z = np.ones((train_set.shape[0], 1))

    best_score = float('inf')
    best_params = {}
    results = []

    total_combinations = len(list(itertools.product(*param_grid.values())))
    for idx, params in enumerate(itertools.product(*param_grid.values())):
        n_estimators, max_depth, min_samples_split, max_iter, n_splits = params
        print(f"Combination: {params}")
        print(f"Progress: {(idx + 1) / total_combinations * 100:.2f}%")

        scores, prev, ptev, oob_scores = [], [], [], []
        kf = KFold(n_splits=n_splits)

        for train_index, test_index in kf.split(X):
            X_train, X_test = X.iloc[train_index], X.iloc[test_index]
            y_train, y_test = y[train_index], y[test_index]
            clusters_train_cv = clusters[train_index]
            clusters_test_cv = pd.Series(clusters[test_index])
            z_train, z_test = z[train_index], z[test_index]

            model = MERF(
                fixed_effects_model=RandomForestRegressor(
                    n_estimators=n_estimators,
                    max_depth=max_depth,
                    min_samples_split=min_samples_split,
                    n_jobs=1,
                    oob_score=True
                ),
                gll_early_stop_threshold=None,
                max_iterations=max_iter
            )

            model.fit(X_train.select_dtypes(include=[np.number]), z_train, pd.Series(clusters_train_cv), y_train)
            y_pred = model.predict(X_test, z_test, clusters_test_cv)
            mse = np.mean((y_pred - y_test) ** 2)
            scores.append(mse)

            total_variance = np.var(y_test)
            random_effect_variance = np.var(y_test - y_pred)
            fixed_effect_variance = total_variance - random_effect_variance

            ptev.append(fixed_effect_variance / total_variance if total_variance > 0 else 0)
            prev.append(random_effect_variance / total_variance if total_variance > 0 else 0)

            forest = model.trained_fe_model
            oob_score = round(forest.oob_score_ * 100, 1)
            oob_scores.append(oob_score)

            print(f"ptev: {np.mean(ptev):.4f}, prev: {np.mean(prev):.4f}, OOB Score: {oob_score:.4f}")

        mean_score = np.mean(scores)
        if mean_score < best_score:
            best_score = mean_score
            best_params = params

        results.append({
            'n_estimators': n_estimators,
            'max_depth': max_depth,
            'min_samples_split': min_samples_split,
            'max_iter': max_iter,
            'n_splits': n_splits,
            'mean_mse_score': mean_score,
            'mean_prev': np.mean(prev),
            'mean_ptev': np.mean(ptev),
            'oob_score': np.mean(oob_scores)
        })

    print("Best parameters:", best_params)
    print("Best score:", best_score)

    results_df = pd.DataFrame(results)
    results_df.to_csv(OUT, index=False)

    print("---------- Run MERF models ----------")
    r2_values, results_df = run_merf_analysis2(
        X, Y, Z, clusters,
        X_new, Y_new, Z_new, clusters_new,
        results_df,
        output_dir, r2_out, r2_adj_out, 
        feature_imp_out, results_filename, time_new
    )

    print("R-squared values:", r2_values)
    print("Results DataFrame:\n", results_df)
    print(f"---------- Done saving Merf output for {key} ----------\n")


In [ ]:
column_sets = {
    "basic": basic,
    "meta_keep": meta_keep,
    "only_grs": only_grs, 
    "only_taxa": only_taxa,
    "only_micom": only_micom,
    "only_pathway": only_pathway,
    "only_metabo" : only_metabo,
    "only_all" : all_col
}

for key, columns in column_sets.items():
    DATA = delta[columns]
    print(f"Subset for {key} created with shape: {DATA.shape}")
    
    OUT = f"/Users/emily/projects/research/Stanislawski/comps/mutli-omic-predictions/play_scripts/2.models/merf_python/may_basic_plus/{key}_delta.csv"
    output_dir = "/Users/emily/projects/research/Stanislawski/comps/mutli-omic-predictions/play_scripts/2.models/merf_python/may_basic_plus/"
    r2_out =  f"{key}_delta_r2.pdf"
    r2_adj_out = f"{key}_delta_r2_adj.pdf"
    feature_imp_out = f"{key}_delta_ft_imp.pdf"
    results_filename = f"/Users/emily/projects/research/Stanislawski/comps/mutli-omic-predictions/play_scripts/2.models/merf_python/may_basic_plus/{key}_delta_april29.csv"

    # Create train and test sets for the current subset
train_set = DATA[DATA[ID_VAR].isin(train_names)]  # Select rows where ID_VAR is in train_names
test_set = DATA[DATA[ID_VAR].isin(test_names)]    # Select rows where ID_VAR is in test_names

# Ensure clusters_train and clusters_test are pandas Series
clusters_train = pd.Series(train_set[ID_VAR])  # Convert to pandas Series
clusters_test = pd.Series(test_set[ID_VAR])    # Convert to pandas Series

# Proceed with MERF analysis for the current subset
print("---------- Select predictors for training set ----------")
X = train_set.drop([BMI_VAR, ID_VAR], axis=1)
Y = train_set[BMI_VAR].to_numpy()  # Convert Y to numeric array
Z = np.ones((train_set.shape[0], 1))  # Create random effects matrix with ones

# Check lengths before fitting
print(f"Length of X: {len(X)}, Length of clusters_train: {len(clusters_train)}, Length of Y: {len(Y)}")
assert len(X) == len(clusters_train), "Length of X does not match length of clusters_train"
assert len(X) == len(Y), "Length of X does not match length of Y"
print("Final columns after drop:", X.columns.to_list())
print("X train values:", train_set[BMI_VAR])

print("---------- Select predictors for test set ----------")
X_new = test_set.drop([BMI_VAR, ID_VAR], axis=1)
X_new = X_new[X.columns]  # Reorder and select columns to match training set
X_new = X_new.astype(X.dtypes)  # Ensure data types match
Y_new = test_set[BMI_VAR].to_numpy()  # Convert Y to numeric array
clusters_new = pd.Series(test_set[ID_VAR])  # Convert to pandas Series
Z_new = np.ones((len(X_new), 1))  # Create random effects matrix with ones
time_new = test_set[TIME_VAR].astype(float).to_numpy()  # Convert time values to numeric arrayo numeric array

# Hyperparameters to tune
param_grid = {
    'n_estimators': [10, 50, 100],
    'max_depth': [None],
    'min_samples_split': [0.05, 0.1, 0.15],
    'max_iter': [2, 10],
    'n_splits': [3, 5, 10] #cross-validation
}
# Create training features
# X = train_set.drop(columns=columns_to_drop, errors='ignore')
y = train_set[[BMI_VAR]]
y = y[BMI_VAR].to_numpy() # Convert Y to numeric array
clusters = train_set[ID_VAR].to_numpy() # Get ID variables
z = np.ones((train_set.shape[0], 1)) # Create random effects matrix with ones

best_score = float('inf')
best_params = {}
results = []  # Initialize a list to store the results of each iteration

# Loop through all possible combinations of parameters
total_combinations = len(list(itertools.product(*param_grid.values())))  # Calculate total combinations
for idx, params in enumerate(itertools.product(*param_grid.values())):
    n_estimators, max_depth, min_samples_split, max_iter, n_splits = params
    
    # Print progress
    progress_percentage = (idx + 1) / total_combinations * 100
    print(f"Combination: {params}\n")
    print(f"Progress: {progress_percentage:.2f}% completed\n")
    scores, prev, ptev, oob_scores = [], [], [], []  # Initialize lists for scores

    # K-fold cross-validation with variable n_splits
    kf = KFold(n_splits=n_splits)
    for train_index, test_index in kf.split(X):
        #print("Train indices:", train_index)
        #print("Test indices:", test_index)
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]  # Use .iloc for row selection
        y_train, y_test = y[train_index], y[test_index]
        clusters_train, clusters_test = clusters[train_index], pd.Series(clusters[test_index])
        z_train, z_test = z[train_index], z[test_index]

        print("Length of clusters_train:", len(clusters_train))
        print("Length of clusters_test:", len(clusters_test))
        #print("Train indices:", train_index)
        #print("Test indices:", test_index)

        model = MERF(
                # Specify the fixed effects model as a Random Forest Regressor
            fixed_effects_model=RandomForestRegressor(
                n_estimators=n_estimators,  # Number of trees in the forest
                max_depth=max_depth,  # Maximum depth of each tree
                min_samples_split=min_samples_split,  # Minimum samples required to split an internal node
                n_jobs=1,  # Number of jobs to run in parallel
                oob_score=True  # Whether to use out-of-bag samples to estimate the R^2 on unseen data
                ),
                # Generalized Linear Model (GLM) early stopping threshold
            gll_early_stop_threshold=None,  # No early stopping threshold set
                # Maximum number of iterations for the MERF algorithm
            max_iterations=max_iter  # Maximum number of iterations to run the MERF algorithm
            )
        model.fit(X_train.select_dtypes(include=[np.number]), z_train, pd.Series(clusters_train), y_train)
        y_pred = model.predict(X_test, z_test, clusters_test)
        scores.append(np.mean((y_pred - y_test) ** 2)) # MSE
        
        # Calculate ptev and prev
        total_variance = np.var(y_test)
        random_effect_variance = np.var(y_test - y_pred)
        fixed_effect_variance = total_variance - random_effect_variance

        ptev.append(np.mean(fixed_effect_variance / total_variance if total_variance > 0 else 0))
        prev.append(np.mean(random_effect_variance / total_variance if total_variance > 0 else 0))

        # Calculate OOB score
        forest = model.trained_fe_model
        oob_score = round(forest.oob_score_ * 100, 1)  # percent variation
        oob_scores.append(oob_score)

        # Print ptev, prev, and OOB score for the current iteration
        print(f"Combination, ptev: {np.mean(ptev):.4f}, prev: {np.mean(prev):.4f}, OOB Score: {oob_score:.4f}")

    # Calculate the mean of the scores for the current combination of parameters
    mean_score = np.mean(scores)
    if mean_score < best_score:
        best_score = mean_score
        best_params = params

    # Append the results of the current iteration to the results list
    result_dict = {
        'n_estimators': n_estimators,
        'max_depth': max_depth,
        'min_samples_split': min_samples_split,
        'max_iter': max_iter,
        'n_splits': n_splits,
        'mean_mse_score': mean_score,
        'mean_prev': np.mean(prev),
        'mean_ptev': np.mean(ptev),
        'oob_score': np.mean(oob_scores)
    }
    results.append(result_dict)

print("Best parameters:", best_params)
print("Best score:", best_score)

# Convert the results list to a DataFrame and save it to a CSV file
results_df = pd.DataFrame(results)
results_df.to_csv(OUT, index=False)

print("---------- Run MERF models ----------")
r2_values, results_df = run_merf_analysis2(
    X, Y, Z, clusters,
    X_new, Y_new, Z_new, clusters_new,
    results_df,
    output_dir, r2_out, r2_adj_out, 
    feature_imp_out, results_filename, time_new)

# Print the R-squared values and the results DataFrame
print("R-squared values:", r2_values)
print("Results DataFrame:\n", results_df)
print("---------- Done saving Merf output ----------")
